In [1]:
import pandas as pd
from datetime import datetime
import requests
import numpy_financial as npf
import numpy as np
import math
import matplotlib.pyplot as plt
import json

In [8]:
data = pd.read_csv("./ОФЗ Анализ - Погашение после 2014 года.csv")
display(data)

,Название,Начало Торгов,Погашение,Купон,Периодичность,ISIN
0,25075,13.02.2012,15.07.2015,"6,88",2,SU25075RMFS1
1,25077,13.02.2012,20.01.2016,7.35,2,SU25077RMFS7
2,25079,13.02.2012,03.06.2015,7,2,SU25079RMFS3
3,25080,25.04.2012,19.04.2017,7.4,2,SU25080RMFS3
4,25081,06.02.2013,31.01.2018,6.2,2,SU25081RMFS4
5,25082,19.06.2013,11.05.2016,6,2,SU25082RMFS4
6,25083,18.10.2017,15.12.2021,7,2,SU25083RMFS5
7,25084,19.02.2020,04.10.2023,5.3,2,SU25084RMFS5
8,25085,22.09.2021,24.09.2025,6.4,2,SU25085RMFS6
9,26202,13.02.2012,17.12.2014,11.2,4,SU26202RMFS0


In [9]:
def numberOfDays(date1, date2, format="%d.%m.%Y"):
  return float((datetime.strptime(date1, format) - datetime.strptime(date2, format)).days)

In [113]:
# Adding date columns
data["start"] = data.apply(lambda row: datetime.strptime(row["Начало Торгов"], "%d.%m.%Y"), axis=1)
data["end"] = data.apply(lambda row: datetime.strptime(row["Погашение"], "%d.%m.%Y"), axis=1)

In [124]:
# Pick date
cur_date_str = "01.10.2024"
cur_date = datetime.strptime(cur_date_str, "%d.%m.%Y")
print(cur_date)
# Select papers
cur_data = data[(data["start"] < cur_date) & (data["end"] > cur_date)]
cur_data.reset_index(drop=True, inplace=True)
display(cur_data)

2024-10-01 00:00:00


,Название,Начало Торгов,Погашение,Купон,Периодичность,ISIN,start,end
0,25085,22.09.2021,24.09.2025,6.4,2,SU25085RMFS6,2021-09-22,2025-09-24
1,26207,22.02.2012,03.02.2027,8.15,2,SU26207RMFS9,2012-02-22,2027-02-03
2,26212,23.01.2013,19.01.2028,7.05,2,SU26212RMFS9,2013-01-23,2028-01-19
3,26218,17.02.2016,17.09.2031,8.5,2,SU26218RMFS6,2016-02-17,2031-09-17
4,26219,06.07.2016,16.09.2026,7.75,2,SU26219RMFS4,2016-07-06,2026-09-16
5,26221,15.03.2017,23.03.2033,7.7,2,SU26221RMFS0,2017-03-15,2033-03-23
6,26222,03.05.2017,16.10.2024,7.1,2,SU26222RMFS8,2017-05-03,2024-10-16
7,26224,07.03.2018,23.05.2029,6.9,2,SU26224RMFS4,2018-03-07,2029-05-23
8,26225,28.02.2018,10.05.2034,7.25,2,SU26225RMFS1,2018-02-28,2034-05-10
9,26226,06.02.2019,07.10.2026,7.95,2,SU26226RMFS9,2019-02-06,2026-10-07


In [125]:
# Calculate yield to maturity
cur_data["days_to_maturity"] = cur_data.apply(lambda row: (row["end"] - cur_date).days, axis=1)
# Convert column data types
cur_data["Купон"] = pd.to_numeric(cur_data["Купон"])
cur_data["Периодичность"] = pd.to_numeric(cur_data["Периодичность"])
sorted_result = cur_data.sort_values("days_to_maturity")
sorted_result["years_to_maturity"] = sorted_result["days_to_maturity"] / 365
sorted_result = sorted_result[sorted_result["days_to_maturity"] >= 365/2]
display(sorted_result)

C:\Users\igor\AppData\Local\Temp\ipykernel_13552\2709368117.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cur_data["days_to_maturity"] = cur_data.apply(lambda row: (row["end"] - cur_date).days, axis=1)
C:\Users\igor\AppData\Local\Temp\ipykernel_13552\2709368117.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cur_data["Купон"] = pd.to_numeric(cur_data["Купон"])
C:\Users\igor\AppData\Local\Temp\ipykernel_13552\2709368117.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

,Название,Начало Торгов,Погашение,Купон,Периодичность,ISIN,start,end,days_to_maturity,years_to_maturity
16,26234,10.06.2020,16.07.2025,4.50,2,SU26234RMFS3,2020-06-10,2025-07-16,288,0.789041
0,25085,22.09.2021,24.09.2025,6.40,2,SU25085RMFS6,2021-09-22,2025-09-24,358,0.980822
11,26229,05.06.2019,12.11.2025,7.15,2,SU26229RMFS3,2019-06-05,2025-11-12,407,1.115068
4,26219,06.07.2016,16.09.2026,7.75,2,SU26219RMFS4,2016-07-06,2026-09-16,715,1.958904
9,26226,06.02.2019,07.10.2026,7.95,2,SU26226RMFS9,2019-02-06,2026-10-07,736,2.016438
1,26207,22.02.2012,03.02.2027,8.15,2,SU26207RMFS9,2012-02-22,2027-02-03,855,2.342466
14,26232,11.12.2019,06.10.2027,6.00,2,SU26232RMFS7,2019-12-11,2027-10-06,1100,3.013699
2,26212,23.01.2013,19.01.2028,7.05,2,SU26212RMFS9,2013-01-23,2028-01-19,1205,3.301370
18,26236,25.11.2020,17.05.2028,5.70,2,SU26236RMFS8,2020-11-25,2028-05-17,1324,3.627397
19,26237,23.06.2021,14.03.2029,6.70,2,SU26237RMFS6,2021-06-23,2029-03-14,1625,4.452055


In [126]:
# Forecasting
def ytm(row):
  return 6.7851 + 6 * 0.0695 + 0.4179 * math.log(row["years_to_maturity"])

In [127]:
sorted_result["ytm"] = sorted_result.apply(ytm, axis=1)
display(sorted_result)

,Название,Начало Торгов,Погашение,Купон,Периодичность,ISIN,start,end,days_to_maturity,years_to_maturity,ytm
16,26234,10.06.2020,16.07.2025,4.50,2,SU26234RMFS3,2020-06-10,2025-07-16,288,0.789041,7.103084
0,25085,22.09.2021,24.09.2025,6.40,2,SU25085RMFS6,2021-09-22,2025-09-24,358,0.980822,7.194008
11,26229,05.06.2019,12.11.2025,7.15,2,SU26229RMFS3,2019-06-05,2025-11-12,407,1.115068,7.247616
4,26219,06.07.2016,16.09.2026,7.75,2,SU26219RMFS4,2016-07-06,2026-09-16,715,1.958904,7.483090
9,26226,06.02.2019,07.10.2026,7.95,2,SU26226RMFS9,2019-02-06,2026-10-07,736,2.016438,7.495187
1,26207,22.02.2012,03.02.2027,8.15,2,SU26207RMFS9,2012-02-22,2027-02-03,855,2.342466,7.557818
14,26232,11.12.2019,06.10.2027,6.00,2,SU26232RMFS7,2019-12-11,2027-10-06,1100,3.013699,7.663114
2,26212,23.01.2013,19.01.2028,7.05,2,SU26212RMFS9,2013-01-23,2028-01-19,1205,3.301370,7.701214
18,26236,25.11.2020,17.05.2028,5.70,2,SU26236RMFS8,2020-11-25,2028-05-17,1324,3.627397,7.740571
19,26237,23.06.2021,14.03.2029,6.70,2,SU26237RMFS6,2021-06-23,2029-03-14,1625,4.452055,7.826178


In [128]:
""" Shit doesnt work
moexDate = datetime.strftime(cur_date, "%Y-%m-%d")
url = "https://iss.moex.com/iss/apps/bondization/yieldscalculator?accint_source=t0&calc_method=by_yield_effective_to_maturity&calc_value=11.34&secid=SU26240RMFS0&sell_date="+ moexDate +"&sell_value=11.34&tradedate=" + moexDate
res = requests.get(url)
result = json.loads(res.text)
print(result)
price = result["calculated"]["PRICE"]
duration = result["calculated"]["DURATIONYEAR"]
print(price, duration)
"""

' Shit doesnt work\nmoexDate = datetime.strftime(cur_date, "%Y-%m-%d")\nurl = "https://iss.moex.com/iss/apps/bondization/yieldscalculator?accint_source=t0&calc_method=by_yield_effective_to_maturity&calc_value=11.34&secid=SU26240RMFS0&sell_date="+ moexDate +"&sell_value=11.34&tradedate=" + moexDate\nres = requests.get(url)\nresult = json.loads(res.text)\nprint(result)\nprice = result["calculated"]["PRICE"]\nduration = result["calculated"]["DURATIONYEAR"]\nprint(price, duration)\n'

In [129]:
""" Shit doesnt work
pv = 0
periods = 12.34
n_per = math.floor(periods)
space = np.arange(0, periods, 0.5)
for i in space:
  payment = 70/2
  pvp = payment / math.pow(1 + 0.1134, i)
  pv += pvp
delta = periods - space[-1]
last_payment = 70 * delta + 1000
print(last_payment)
pv += last_payment / math.pow(1 + 0.1134, periods)
print(pv)
"""

' Shit doesnt work\npv = 0\nperiods = 12.34\nn_per = math.floor(periods)\nspace = np.arange(0, periods, 0.5)\nfor i in space:\n  payment = 70/2\n  pvp = payment / math.pow(1 + 0.1134, i)\n  pv += pvp\ndelta = periods - space[-1]\nlast_payment = 70 * delta + 1000\nprint(last_payment)\npv += last_payment / math.pow(1 + 0.1134, periods)\nprint(pv)\n'

In [130]:
""" Sort of works """
ppv = 35.5 * (1  - math.pow(1.055, -17.13*2))/0.1134*2 + 1000 / math.pow(1.056, 17.13*2)
print(ppv)

680.72162629486


In [131]:
def present_value(row):
  coupon = row["Купон"]
  duration = row["years_to_maturity"]
  frequency = row["Периодичность"]
  coupon_value = 1000 / 100 * coupon / frequency
  number_of_periods = duration * frequency
  rate_of_return = math.pow(1 + row["ytm"]/100, 1 / frequency)
  cumulative_coupon_value = coupon_value * (1 - math.pow(rate_of_return, -number_of_periods))/ (rate_of_return -1)
  final_discounted_value = 1000 / math.pow(rate_of_return, number_of_periods)
  present_value = cumulative_coupon_value + final_discounted_value
  return present_value

In [132]:
sorted_result["pv"] = sorted_result.apply(present_value, axis=1)
sorted_result["Price"] = sorted_result["pv"] / 10
display(sorted_result)

,Название,Начало Торгов,Погашение,Купон,Периодичность,ISIN,start,end,days_to_maturity,years_to_maturity,ytm,pv,Price
16,26234,10.06.2020,16.07.2025,4.50,2,SU26234RMFS3,2020-06-10,2025-07-16,288,0.789041,7.103084,981.267656,98.126766
0,25085,22.09.2021,24.09.2025,6.40,2,SU25085RMFS6,2021-09-22,2025-09-24,358,0.980822,7.194008,993.765660,99.376566
11,26229,05.06.2019,12.11.2025,7.15,2,SU26229RMFS3,2019-06-05,2025-11-12,407,1.115068,7.247616,1000.307251,100.030725
4,26219,06.07.2016,16.09.2026,7.75,2,SU26219RMFS4,2016-07-06,2026-09-16,715,1.958904,7.483090,1007.209987,100.720999
9,26226,06.02.2019,07.10.2026,7.95,2,SU26226RMFS9,2019-02-06,2026-10-07,736,2.016438,7.495187,1010.876060,101.087606
1,26207,22.02.2012,03.02.2027,8.15,2,SU26207RMFS9,2012-02-22,2027-02-03,855,2.342466,7.557818,1015.432162,101.543216
14,26232,11.12.2019,06.10.2027,6.00,2,SU26232RMFS7,2019-12-11,2027-10-06,1100,3.013699,7.663114,959.639556,95.963956
2,26212,23.01.2013,19.01.2028,7.05,2,SU26212RMFS9,2013-01-23,2028-01-19,1205,3.301370,7.701214,985.387971,98.538797
18,26236,25.11.2020,17.05.2028,5.70,2,SU26236RMFS8,2020-11-25,2028-05-17,1324,3.627397,7.740571,940.846407,94.084641
19,26237,23.06.2021,14.03.2029,6.70,2,SU26237RMFS6,2021-06-23,2029-03-14,1625,4.452055,7.826178,963.673513,96.367351


In [133]:
final_result = sorted_result[["Название", "Купон", "ISIN", "years_to_maturity", "ytm", "Price"]]
display(final_result)

,Название,Купон,ISIN,years_to_maturity,ytm,Price
16,26234,4.50,SU26234RMFS3,0.789041,7.103084,98.126766
0,25085,6.40,SU25085RMFS6,0.980822,7.194008,99.376566
11,26229,7.15,SU26229RMFS3,1.115068,7.247616,100.030725
4,26219,7.75,SU26219RMFS4,1.958904,7.483090,100.720999
9,26226,7.95,SU26226RMFS9,2.016438,7.495187,101.087606
1,26207,8.15,SU26207RMFS9,2.342466,7.557818,101.543216
14,26232,6.00,SU26232RMFS7,3.013699,7.663114,95.963956
2,26212,7.05,SU26212RMFS9,3.301370,7.701214,98.538797
18,26236,5.70,SU26236RMFS8,3.627397,7.740571,94.084641
19,26237,6.70,SU26237RMFS6,4.452055,7.826178,96.367351
